# Tabel of Contents

1. 데이터 탐색하기와 전처리
2. 사용자의 명시적/암묵적 평가
3. Matrix Factorization(MF)
4. CSR(Compressed Sparse Row) Matrix
5. MF 모델 학습하기
6. 비슷한 아티스트 찾기 + 유저에게 추천하기
7. 프로젝트 - MovieLens 영화 추천 실습

link: http://www.kocca.kr/insight/vol05/vol05_04.pdf

tsv 포맷 열기위해서 seperator '\t' 설정

In [27]:
import pandas as pd
import os

fname = os.getenv("HOME") + '/aiffel/recommendata_iu/data/lastfm-dataset-360k/usersha1-artmbid-artname-plays.tsv'
col_names=['user_id', 'artist_MBID', 'artist','play']
data = pd.read_csv(fname, sep='\t', names=col_names)
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


원하는 컬럼만 따로 선택

In [22]:
using_cols=['user_id','artist','play']
data=data[using_cols]
data.head()

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [23]:
data['artist'] = data['artist'].str.lower()
data.head()

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


첫 번째 유저 정보 출력

In [24]:
condition = (data['user_id']==data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


### 데이터 탐색
```df.nunique()```

In [29]:
# 유저수
data['user_id'].nunique()

358868

In [31]:
# 아티스트 수
data['artist'].nunique()

292363

In [33]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(10)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
Name: user_id, dtype: int64

In [35]:
# 유저별 몇명의 아스트를 듣고 있느니 에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [36]:
# 유저별 play 횟수 중앙값 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

In [37]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist_MBID,artist,play
17535650,"sep 20, 2008",7ffd711a-b34d-4739-8aab-25e045c246da,turbostaat,12
17535651,"sep 20, 2008",9201190d-409f-426b-9339-9bd7492443e2,cuba missouri,11
17535652,"sep 20, 2008",e7cf7ff9-ed2f-4315-aca8-bcbd3b2bfa71,little man tate,11
17535653,"sep 20, 2008",f6f2326f-6b25-4170-b89d-e235b25508e8,sigur rós,10
17535654,"sep 20, 2008",40f5d9e4-2de7-4f2d-ad41-e31a9a9fea27,the smiths,10
0,zimin,NaN,black eyed peas,30
1,zimin,NaN,maroon5,30
2,zimin,NaN,jason mraz,30
3,zimin,NaN,coldplay,30
4,zimin,NaN,beyoncé,30


In [38]:
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

user_to_idx = {v: k for k,v in enumerate(user_unique)}
artist_to_idx = {v: k for k,v in enumerate(artist_unique)}

In [39]:
print(user_to_idx['zimin'])
print(artist_to_idx['black eyed peas'])

358868
376


In [40]:
# indexing 을 통해 테이터 칼럼 내 값을 바꾸는 코드

# user_to_idx.get 을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다.
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거합니다

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')
    
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist']=temp_artist_data
else:
    print('artist column indexing FAIL!!!')

data

user_id column indexing OK!!!
artist column indexing OK!!


,user_id,artist_MBID,artist,play
0,0,3bd73256-3905-4f3a-97e2-8b341527f805,0,2137
1,0,f2fb0ff0-5679-42ec-a55c-15109ce6e320,1,1099
2,0,b3ae82c2-e60b-4551-a76d-6620f1b456aa,2,897
3,0,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,3,717
4,0,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,4,706
...,...,...,...,...
0,358868,NaN,376,30
1,358868,NaN,271017,30
2,358868,NaN,3746,30
3,358868,NaN,62,30


In [41]:
# 1회만 play 한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')

147740,17535660
Ratio of only_one over all data is 0.84%


### CSR(Compressed Sparse Row) Matrix

'indprt' Index pointers is linked list of pointers to
'indices'(column index Pointers)...
link: https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005

In [43]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape=(num_user, num_artist))
csr_data

<358869x292364 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535585 stored elements in Compressed Sparse Row format>

Matrix Fatorization -> Implicit(암묵적) dataset package
als(AlternatingLeastSquares) 모델 : 두 feature matrix 중 한쪽을 고정시고 다른 쪽을 학습하는 방식을 번갈아 수행함

In [44]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [45]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01,
                                    use_gpu=False,
                                    dtype=np.float32)

In [46]:
# als 모델은 input으로 (item X user 꼴의 matrix를 때문에 닸ranspose 해줍니다)
csr_data_transpose = csr_data.T
csr_data_transpose

<292364x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535585 stored elements in Compressed Sparse Column format>

In [47]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [48]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [49]:
zimin_vector

array([-5.58084488e-01, -1.38451368e-01, -2.37757847e-01,  8.32318783e-01,
       -5.70116527e-02, -1.35720563e+00, -2.77664572e-01, -6.30040169e-01,
       -1.45044699e-01, -1.01854146e+00,  4.58521307e-01, -1.06646597e+00,
       -7.97158387e-03,  1.31048059e+00,  3.75720978e-01, -3.76204789e-01,
       -1.14018805e-01,  1.11928098e-01,  8.25471759e-01, -7.59567507e-03,
        1.55405772e+00,  3.49292427e-01, -1.00414634e-01,  5.20219028e-01,
       -3.22346926e-01, -1.04816461e+00,  1.10404216e-01, -4.93673414e-01,
        4.41059992e-02, -8.32776964e-01,  2.71921664e-01,  1.71762764e+00,
       -6.68184400e-01, -4.73797992e-02, -2.94839472e-01,  8.03386331e-01,
       -6.52597308e-01, -1.14947832e+00, -1.74510896e+00, -4.52875465e-01,
       -7.11654872e-02, -1.27276218e+00,  4.34712887e-01,  6.23202384e-01,
        1.90261424e-01,  1.28737614e-01,  6.73704967e-02,  5.83447158e-01,
        5.11845648e-01,  5.34697950e-01,  1.90781400e-01, -4.82020378e-01,
       -9.69070733e-01, -

In [50]:
black_eyed_peas_vector

array([-0.00180914,  0.02359356, -0.00662939,  0.00653167,  0.01748352,
        0.00199991, -0.01224809, -0.00751665,  0.00158728, -0.00553667,
        0.00779487, -0.00515218,  0.00535332,  0.0123032 ,  0.01012068,
       -0.00313465, -0.01363786,  0.01954222,  0.01496124, -0.00410942,
        0.02972317,  0.00473042, -0.00124717,  0.01315532,  0.0055931 ,
       -0.01136929,  0.01020056, -0.00328502,  0.00919188, -0.00367534,
        0.00474358,  0.01867747, -0.00164955,  0.00038741,  0.00595106,
        0.01653088, -0.00325221, -0.01138331, -0.0058842 ,  0.01511716,
        0.0090636 , -0.01085387,  0.01367447,  0.01457198,  0.00945232,
        0.01530872,  0.01865887,  0.02127362,  0.01183216,  0.0131022 ,
        0.02224982,  0.00803497, -0.00751463,  0.00323679,  0.01564962,
        0.00597247, -0.00731711,  0.00019252, -0.01721236,  0.00827466,
        0.01930525,  0.00877372,  0.01899979,  0.01202327,  0.01273281,
        0.00926096,  0.01641816,  0.00815576,  0.00355271,  0.00

In [51]:
# zimin, black_eyed_peas 내적
np.dot(zimin_vector, black_eyed_peas_vector)

0.5124239

In [55]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3008922

### 비슷한 아티스트 찾기 + 유저에게 추천하기
```AlternatingLeastSquares.similar_items()```

In [56]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.9999999),
 (277, 0.98835284),
 (5, 0.97690046),
 (28, 0.9720171),
 (217, 0.9694139),
 (473, 0.96598005),
 (490, 0.9607386),
 (247, 0.9589004),
 (910, 0.9541248),
 (418, 0.9532959),
 (694, 0.9497932),
 (782, 0.9425723),
 (268, 0.9386042),
 (1018, 0.9379625),
 (531, 0.93480754)]

In [58]:
#artist_to_idx를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'the killers',
 'radiohead',
 'placebo',
 'oasis',
 'the beatles',
 'nirvana',
 'u2',
 'foo fighters',
 'the white stripes',
 'pink floyd',
 'the smashing pumpkins',
 'depeche mode']

In [59]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [62]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'katy perry',
 'rihanna',
 'beyoncé',
 'the pussycat dolls',
 'kelly clarkson',
 'christina aguilera',
 'justin timberlake',
 'leona lewis']

__유저에게 아티스트 추천하기__
```
AlternatingLeastSquares.recommmend()
AlternatingLeastSquares.filter_already_liked_items()
```

In [63]:
user = user_to_idx['zimin']
# recommend -> user*item CSR Matrix
artist_recommended = als_model.recommend(user,csr_data, N=20, 
                                         filter_already_liked_items=True)
artist_recommended

[(350, 0.44454262),
 (369, 0.43977642),
 (550, 0.43252516),
 (627, 0.4151128),
 (1800, 0.41475344),
 (2249, 0.414375),
 (274, 0.3913468),
 (354, 0.38966948),
 (382, 0.38825563),
 (355, 0.3849309),
 (391, 0.38430375),
 (5556, 0.3738802),
 (409, 0.3715177),
 (564, 0.3661566),
 (901, 0.3645937),
 (618, 0.36204734),
 (358, 0.35461217),
 (24, 0.35427287),
 (724, 0.34940523),
 (944, 0.34871125)]

In [64]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'maroon 5',
 'lady gaga',
 'katy perry',
 'michael jackson',
 'nelly furtado',
 'mika',
 'madonna',
 'christina aguilera',
 'timbaland',
 'amy winehouse',
 'kanye west',
 'pink',
 'the pussycat dolls',
 'kelly clarkson',
 'jack johnson',
 'lily allen',
 'avril lavigne']

In [65]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [68]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.219924769100042),
 ('black eyed peas', 0.14042903441010368),
 ('jason mraz', 0.04668546196520325),
 ('coldplay', 0.03947152519784778),
 ('maroon5', -2.9961741062687594e-06)]